In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
import pandas as pd
import sys
from vllm import LLM, SamplingParams
sys.path.append("../")
# import utils

model_path = '../Models/llama-2-7b-chat/'
model = LLM(model=model_path, gpu_memory_utilization=0.5)

INFO 12-02 22:25:20 llm_engine.py:72] Initializing an LLM engine with config: model='../Models/llama-2-7b-chat/', tokenizer='../Models/llama-2-7b-chat/', tokenizer_mode=auto, revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 12-02 22:25:20 tokenizer.py:30] For some LLaMA V1 models, initializing the fast tokenizer may take a long time. To reduce the initialization time, consider using 'hf-internal-testing/llama-tokenizer' instead of the original tokenizer.
INFO 12-02 22:25:46 llm_engine.py:205] # GPU blocks: 3408, # CPU blocks: 512


In [2]:

# disease_dict_EHR = pd.read_csv('./mimic-iii/disease_dict_EHR.csv')
disease_dict_EHR = pd.read_csv('./mimic-iii/D_ICD_DIAGNOSES.csv')
# add a column 'concise_title' with null values
disease_dict_EHR['CONCISE_TITLE'] = None

# rename all long_title to concise disease name
batch = []
for i in range(len(disease_dict_EHR)):
    long_title = disease_dict_EHR.loc[i, 'LONG_TITLE']

    prompts = [f"Rename 'Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease' to be concise disease name: 'Malignant Hypertensive End Stage Renal Disease'\
    \
    Rename 'Infection and inflammatory reaction due to other vascular device, implant, and graft' to be concise disease name: 'Vascular Device Infection'\
    \
    Rename 'Subendocardial infarction, initial episode of care' to be concise disease name: 'Initial Subendocardial Infarction'\
    \
    Rename 'Surgical operation with anastomosis, bypass, or graft, with natural or artificial tissues used as implant causing abnormal patient reaction, or later complication, without mention of misadventure at time of operation' to be concise disease name: 'Implant Complication after Surgical Anastomosis' \
    \
    Rename 'Intestinal infection due to Clostridium difficile' to be concise disease name: Clostridium difficile Enteritis'\
    \
    Rename 'Methicillin susceptible Staphylococcus aureus in conditions classified elsewhere and of unspecified site' to be concise disease name: 'Methicillin-Susceptible Staphylococcus Aureus Infection'\
    \
    Rename 'Other specified bacterial infections in conditions classified elsewhere and of unspecified site, other anaerobes' to be concise disease name: 'Anaerobe Infection'\
    \
    Rename 'Diabetes with ophthalmic manifestations, type II or unspecified type, uncontrolled' to be concise disease name: 'Type II Diabetic Retinopathy'\
    \
    Rename 'Other specified disorders of adrenal glands' to be concise disease name: 'Adrenal Gland Disorder'\
    \
    Rename 'Other causes of encephalitis and encephalomyelitis' to be concise disease name: 'Encephalitis and Encephalomyelitis'\
    \
    Rename '{long_title}' to be concise disease name: \
    "]
    batch.extend(prompts)


sampling_params = SamplingParams(temperature=0, presence_penalty=1, max_tokens=50, stop=["\n", "Rename"])
msg = model.generate(batch, sampling_params)
msg = list(map(lambda x: x.outputs[0].text.split('Rename')[0].split('\n')[0].strip("\'\n '"), msg))
# for m in msg:
#     print(long_title)
#     print(m)
for i in range(len(disease_dict_EHR)):
    disease_dict_EHR.loc[i, 'CONCISE_TITLE'] = msg[i]
    
# disease_dict_EHR.to_csv('./mimic-iii/disease_dict_EHR.csv', index=False)
disease_dict_EHR.to_csv('./mimic-iii/D_ICD_DIAGNOSES.csv', index=False)
print(disease_dict_EHR.SHORT_TITLE.str.len().mean().__round__(1), 
      disease_dict_EHR.LONG_TITLE.str.len().mean().__round__(1), 
      disease_dict_EHR.CONCISE_TITLE.str.len().mean().__round__(1))
# 计算disease_dict_EHR.CONCISE_TITLE的平均长度
# 21.2 44.9 29.5  现有的
# 21.2 44.9 26.5  去掉 Unspecified Site
# 21.2 44.9 24.8  增加了糖尿病的例子
# 21.2 44.9 23.9  增加了肾病的例子
# 21.2 44.9 23.6  去掉了一个other

Processed prompts: 100%|██████████| 14567/14567 [09:42<00:00, 24.99it/s] 


21.7 54.6 25.2


In [3]:
procedure_dict = pd.read_csv('./mimic-iii/D_ICD_PROCEDURES.csv')
# add a column 'concise_title' with null values
procedure_dict['CONCISE_TITLE'] = None

# rename all long_title to concise procedure name
batch = []
for i in range(len(procedure_dict)):
    long_title = procedure_dict.loc[i, 'LONG_TITLE']
# Lumbar sympathectomy
# Lumbar sympathectomy
    prompts = [f"Rename 'Microscopic examination of specimen from trachea, bronchus, pleura, lung, and other thoracic specimen, and of sputum, cell block and Papanicolaou smear' to be concise procedure name: 'Thoracic Specimen Microscopy'\
    \
    Rename 'Reconstruction of eyelid involving lid margin, partial-thickness' to be concise procedure name: 'Eyelid Margin Reconstruction'\
    \
    Rename 'Other reconstruction of eyelid with flaps or grafts' to be concise procedure name: 'Eyelid Reconstruction with Flaps or Grafts'\
    \
    Rename 'Lumbar sympathectomy' to be concise procedure name: 'Lumbar Sympathectomy'\
    \
    Rename 'Microscopic examination of specimen from other site, parasitology' to be concise procedure name: 'Parasitology'\
    \
    Rename 'Cardiopulmonary resuscitation, not otherwise specified' to be concise procedure name: 'CPR'\
    \
    Rename '{long_title}' to be concise procedure name: "]
    batch.extend(prompts)

sampling_params = SamplingParams(temperature=0, presence_penalty=1, max_tokens=50, stop=["\n", "Rename"])
msg = model.generate(batch, sampling_params)
msg = list(map(lambda x: x.outputs[0].text.split('Rename')[0].split('\n')[0].strip("\'\n '"), msg))
# for idx, m in enumerate(msg):
#     print(procedure_dict.loc[idx, 'LONG_TITLE'])
#     print(m)
#     print('\n')
for i in range(len(procedure_dict)):
    procedure_dict.loc[i, 'CONCISE_TITLE'] = msg[i]

procedure_dict.to_csv('./mimic-iii/D_ICD_PROCEDURES.csv', index=False)
print(procedure_dict.SHORT_TITLE.str.len().mean().__round__(1), 
      procedure_dict.LONG_TITLE.str.len().mean().__round__(1), 
      procedure_dict.CONCISE_TITLE.str.len().mean().__round__(1))

Processed prompts: 100%|██████████| 3882/3882 [01:27<00:00, 44.47it/s] 


21.6 41.0 22.2


In [53]:
batch

["Rename 'Microscopic examination of specimen from trachea, bronchus, pleura, lung, and other thoracic specimen, and of sputum, cell block and Papanicolaou smear' to be concise procedure name: 'Thoracic Specimen Microscopy'        Rename 'Reconstruction of eyelid involving lid margin, partial-thickness' to be concise procedure name: 'Eyelid Margin Reconstruction'    Rename 'Canthotomy' to be concise procedure name: ",
 "Rename 'Microscopic examination of specimen from trachea, bronchus, pleura, lung, and other thoracic specimen, and of sputum, cell block and Papanicolaou smear' to be concise procedure name: 'Thoracic Specimen Microscopy'        Rename 'Reconstruction of eyelid involving lid margin, partial-thickness' to be concise procedure name: 'Eyelid Margin Reconstruction'    Rename 'Blepharorrhaphy' to be concise procedure name: ",
 "Rename 'Microscopic examination of specimen from trachea, bronchus, pleura, lung, and other thoracic specimen, and of sputum, cell block and Papanico

In [52]:
procedure_dict

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE,CONCISE_TITLE
0,264,851,Canthotomy,Canthotomy,None
1,265,852,Blepharorrhaphy,Blepharorrhaphy,None
2,266,859,Adjust lid position NEC,Other adjustment of lid position,None
3,267,861,Lid reconst w skin graft,Reconstruction of eyelid with skin flap or graft,None
4,268,862,Lid reconst w muc graft,Reconstruction of eyelid with mucous membrane ...,None
...,...,...,...,...,...
3877,3344,9959,Vaccination/innocula NEC,Other vaccination and inoculation,None
3878,3345,9960,Cardiopulm resuscita NOS,"Cardiopulmonary resuscitation, not otherwise s...",None
3879,3346,9961,Atrial cardioversion,Atrial cardioversion,None
3880,3347,9962,Heart countershock NEC,Other electric countershock of heart,None
